In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px

In [ ]:
# import csv 

average_data = pd.read_csv("advanced_ml/average_player.csv")
average_player_df = average_data.set_index('Unnamed: 0')
average_player_df.index.name = None
average_player_df.head()

 class one df has 720 players, looking at the data, these players tend to be average nba players. 
 I am planning to run an elbow curve and add clusters within this DF 

In [ ]:
# create average columns df for the machine learning model

average = [average_player_df["G"],average_player_df["MP"], average_player_df["PER"],average_player_df["BPM"],average_player_df["VORP"]]
average_df = pd.concat(average, axis=1)
average_df.head()

In [ ]:
# scale the average df

average_scaled = StandardScaler().fit_transform(average_df)
print(average_scaled[0:5])

In [ ]:
# Initialize PCA model
pca = PCA(n_components=3)

In [ ]:
# Get 3 principal components for the data.
average_pca = pca.fit_transform(average_scaled)

In [ ]:
# create PCA df
df_average_pca = pd.DataFrame(
    data = average_pca, columns = ["principal component 1", "principal component 2", "principal component 3"], index=average_player_df.index)
df_average_pca.head()

In [ ]:
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_average_pca)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

In [ ]:
# make "Player" and "Year" its own df

player = [average_player_df["Player"],average_player_df["Season"]]
player_name = pd.concat(player, axis=1)
player_name.head()

In [ ]:
# Elbow curve is 2 so we will have two clusters

# Initialize the K-means model
model = KMeans(n_clusters=2, random_state=0)

# Fit the model
model.fit(df_average_pca)

# Predict clusters
predictions = model.predict(df_average_pca)

# Add the predicted class columns
player_name["class"] = model.labels_
player_name.head()

In [ ]:
#sort by class

sorted_class = player_name.sort_values(["class"])
class_one = sorted_class[(sorted_class["class"] == 0)]
class_two = sorted_class[(sorted_class["class"] == 1)]
class_one.head()

In [ ]:
# class 1 df

df_one = pd.concat([class_one, average_df], axis=1)
top_average = pd.DataFrame(index=average_player_df.index)
top_average = df_one.dropna()
top_average.count()
top_average.to_csv('average_advanced_players.csv')

In [ ]:
# class 2 df

df_two = pd.concat([class_two, average_df], axis=1)
bottom_average = pd.DataFrame(index=average_player_df.index)
bottom_average = df_two.dropna()
bottom_average.count()
bottom_average.to_csv('slightly_below_advanced_players.csv')